In [94]:
from TTS_DataLoader import LjSpeechDataset, get_data_loader
from datasets import load_dataset
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
hf_dataset = load_dataset('keithito/lj_speech')['train']
hf_dataset.set_format(type="torch", columns=["audio"], output_all_columns=True)
hf_dataset[0]

{'audio': {'path': '/home/sid/.cache/huggingface/datasets/downloads/extracted/b4bfb01a5a85fb305eaf2cfa68858da969c972369500b02a6eefaf13eb08bb5a/LJSpeech-1.1/wavs/LJ001-0001.wav',
  'array': tensor([-7.3242e-04, -7.6294e-04, -6.4087e-04,  ...,  7.3242e-04,
           2.1362e-04,  6.1035e-05]),
  'sampling_rate': tensor(22050)},
 'id': 'LJ001-0001',
 'file': '/home/sid/.cache/huggingface/datasets/downloads/extracted/b4bfb01a5a85fb305eaf2cfa68858da969c972369500b02a6eefaf13eb08bb5a/LJSpeech-1.1/wavs/LJ001-0001.wav',
 'text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition',
 'normalized_text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition'}

In [ ]:
ds = LjSpeechDataset(hf_dataset)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch

In [ ]:
def speech_collate_fn(batch):
    text_seqs, mel_specs = zip(*batch)
    text_seq_lens = [text_seq.shape[-1] for text_seq in text_seqs] # batch first
    mel_specs_t = []
    mel_spec_lens = []
    for mel_spec in mel_specs:
        mel_specs_t.append(mel_spec.T)
        mel_spec_lens.append(mel_spec.shape[-1])

    
    padded_text_seqs = pad_sequence(text_seqs, batch_first=True, padding_value=0)
    padded_mel_specs = pad_sequence(mel_specs_t, batch_first=True, padding_value=0)
    return padded_text_seqs, text_seq_lens, padded_mel_specs.permute(0, 2, 1), mel_spec_lens

In [117]:
data_loader = DataLoader(ds, batch_size=16, shuffle=True, collate_fn=speech_collate_fn)
padded_text_seqs, text_seq_lens, padded_mel_specs, mel_spec_lens = next(iter(data_loader))
print(padded_text_seqs.shape, len(text_seq_lens), padded_mel_specs.shape, len(mel_spec_lens))
# print(padded_text_seqs.shape) # torch.Size([1, 35]) B x MAX_TEXT_IN_BATCH
# print(padded_mel_specs.shape) # torch.Size([1, 128, 225]) B x num_mels x MAX_MEL_IN_BATCH

torch.Size([16, 170]) 16 torch.Size([16, 128, 1097]) 16
